In [1]:
import os
import pandas as pd
# requires dammit env
# source activate dammit
from dammit.fileio.gff3 import GFF3Parser

# All contigs quantified by salmon

In [2]:
table = pd.read_csv("/Users/johnsolk/Documents/UCDavis/Whitehead/RNAseq_15killifish/salmon_denovo_by_species/L_goodei/L_goodei_transfer_2.quant/quant.sf",sep="\t")
table.head()

,Name,Length,EffectiveLength,TPM,NumReads
0,TRINITY_DN2202_c0_g1_i1,278,70.590,0.000000,0.000000
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000
2,TRINITY_DN2271_c0_g1_i1,305,88.409,0.000000,0.000000
3,TRINITY_DN2271_c1_g1_i1,525,279.936,0.000000,0.000000
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129


In [3]:
table.shape

(385474, 5)

In [4]:
contig_names = pd.DataFrame(table.Name.str.split("_",5).tolist(),columns = ['Trinity','Contig','Component','Gene','Isoform'])

In [5]:
contig_names.head()

,Trinity,Contig,Component,Gene,Isoform
0,TRINITY,DN2202,c0,g1,i1
1,TRINITY,DN2202,c1,g1,i1
2,TRINITY,DN2271,c0,g1,i1
3,TRINITY,DN2271,c1,g1,i1
4,TRINITY,DN2268,c0,g1,i1


In [6]:
contig_names2 = pd.DataFrame()
contig_names2['Name'] = contig_names[['Trinity','Contig','Component','Gene','Isoform']].apply(lambda x: '_'.join(x),axis=1)

In [7]:
contig_names2['Gene'] = contig_names[['Trinity','Contig','Component','Gene']].apply(lambda x: '_'.join(x),axis=1)

In [8]:
contig_names2.head()

,Name,Gene
0,TRINITY_DN2202_c0_g1_i1,TRINITY_DN2202_c0_g1
1,TRINITY_DN2202_c1_g1_i1,TRINITY_DN2202_c1_g1
2,TRINITY_DN2271_c0_g1_i1,TRINITY_DN2271_c0_g1
3,TRINITY_DN2271_c1_g1_i1,TRINITY_DN2271_c1_g1
4,TRINITY_DN2268_c0_g1_i1,TRINITY_DN2268_c0_g1


In [9]:
contig_names2.shape

(385474, 2)

# Trinity "genes"

In [10]:
contig_names2['Gene'].nunique()

266317

In [11]:
table_2 = pd.merge(table,contig_names2,on='Name')

In [12]:
table_2.shape

(385474, 6)

In [13]:
table_2.head()

,Name,Length,EffectiveLength,TPM,NumReads,Gene
0,TRINITY_DN2202_c0_g1_i1,278,70.590,0.000000,0.000000,TRINITY_DN2202_c0_g1
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000,TRINITY_DN2202_c1_g1
2,TRINITY_DN2271_c0_g1_i1,305,88.409,0.000000,0.000000,TRINITY_DN2271_c0_g1
3,TRINITY_DN2271_c1_g1_i1,525,279.936,0.000000,0.000000,TRINITY_DN2271_c1_g1
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1


# Removed contigs with no expression

In [14]:
table_filt = table_2.loc[table_2['NumReads'] > 0.0]

In [15]:
table_filt.shape

(202604, 6)

In [16]:
table_filt.head()

,Name,Length,EffectiveLength,TPM,NumReads,Gene
1,TRINITY_DN2202_c1_g1_i1,290,78.150,1.087338,1.000000,TRINITY_DN2202_c1_g1
4,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1
5,TRINITY_DN2201_c0_g1_i1,211,38.753,2.192735,1.000000,TRINITY_DN2201_c0_g1
7,TRINITY_DN2275_c0_g1_i1,309,91.270,0.931033,1.000000,TRINITY_DN2275_c0_g1
9,TRINITY_DN2272_c0_g1_i1,374,142.516,0.596254,1.000000,TRINITY_DN2272_c0_g1


# Trinity "genes", contigs removed with no expression

In [17]:
table_filt['Gene'].nunique()

142016

In [18]:
name = "/Users/johnsolk/Documents/UCDavis/Whitehead/gff_annotations/L_goodei.trinity_out.Trinity.fasta.dammit.gff3"
conversion = "/Users/johnsolk/Documents/UCDavis/Whitehead/dammit_conversions/L_goodei.trinity_out.Trinity.fasta.dammit.namemap.csv"
Fhet_counts = "/Users/johnsolk/Documents/UCDavis/Whitehead/Fhet_counts/L_goodei_genenames.csv"

In [19]:
annotations = GFF3Parser(filename=name).read()
annotations = annotations.dropna(subset=['Name'])

/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):
/Users/johnsolk/anaconda3/lib/python3.6/site-packages/dammit/fileio/gff3.py:73: ParserWarning: Both a converter and dtype were specified for column attributes - only the converter will be used
  dtype=dict(self.columns)):


In [20]:
annotations["length"] = annotations["end"].subtract(annotations["start"], fill_value=0)

# Annotations in all contigs (one annotation per contig)

In [21]:
pickonename = annotations.sort_values(by=['seqid', 'score'], ascending=True).query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','Note','database','Dbxref','start','end','length']]
pickonename = pickonename.dropna(axis=0,how="all")
print('Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):')
print(pickonename.shape)

Number of contigs with annotations (one annotation/contig, sorted by E-value < 1e-05 and picked the lowest):
(138485, 8)


In [22]:
pickonename.head()

,seqid,Name,Note,database,Dbxref,start,end,length
110,Transcript_10000,ENSXMAP00000013838,NaN,OrthoDB,NaN,1,382,381
2,Transcript_100000,ENSXMAP00000016367,NaN,OrthoDB,NaN,163,1480,1317
9,Transcript_100001,ENSXMAP00000002468,NaN,OrthoDB,NaN,367,739,372
16,Transcript_100002,ENSXMAP00000002468,NaN,OrthoDB,NaN,19,298,279
24,Transcript_100003,ENSXMAP00000017080,NaN,OrthoDB,NaN,285,1002,717


In [23]:
conversions = pd.read_csv(conversion)

In [24]:
conversions['Name'], conversions['info'] = conversions['original'].str.split(' ', 1).str
conversions = conversions[['Name','renamed']]
conversions.columns = ['Name','seqid']
conversions.head()

,Name,seqid
0,TRINITY_DN2202_c0_g1_i1,Transcript_0
1,TRINITY_DN2202_c1_g1_i1,Transcript_1
2,TRINITY_DN2271_c0_g1_i1,Transcript_2
3,TRINITY_DN2271_c1_g1_i1,Transcript_3
4,TRINITY_DN2268_c0_g1_i1,Transcript_4


In [25]:
table_filt.shape

(202604, 6)

In [175]:
merged_table = pd.merge(table_filt,conversions,on="Name")
merged_table.shape

(202604, 7)

In [176]:
merged_table['Gene'].nunique()

142016

# Annotations, contigs removed with no expression

In [177]:
merged_table = pd.merge(merged_table,pickonename,on="seqid")

In [178]:
merged_table.shape

(87271, 14)

In [179]:
merged_table.head()

,Name_x,Length,EffectiveLength,TPM,NumReads,Gene,seqid,Name_y,Note,database,Dbxref,start,end,length
0,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187,271,84
1,TRINITY_DN2223_c0_g1_i1,272,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2,272,270
2,TRINITY_DN2294_c0_g1_i1,691,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1,85,84
3,TRINITY_DN2293_c0_g1_i1,646,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395,482,87
4,TRINITY_DN73018_c0_g1_i1,425,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0,425,425


# Unique gene names in filtered contigs

In [182]:
print(len(merged_table.Name_y.unique()))

36695


# Unique Trinity "genes" in filtered, annotated contigs

In [183]:
print(len(merged_table.Gene.unique()))

43627


# *Fundulus heteroclitus* (RefSeq) annotations

In [184]:
fhet = annotations[annotations['Name'].str.startswith("gi")]
fhet_filtered = fhet.query('score < 1e-05').drop_duplicates(subset='seqid')[['seqid', 'Name','start','end','length']]
print("Number of unique Fhet gene names:")
print(len(fhet_filtered.Name.unique()))

Number of unique Fhet gene names:
22631


In [185]:
fhet_filtered.head()

,seqid,Name,start,end,length
25,Transcript_100003,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,95,334,239
34,Transcript_100004,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,0,83,83
50,Transcript_100006,gi|831503966|ref|XP_012711679.1| PREDICTED: pi...,413,433,20
68,Transcript_100007,gi|831503966|ref|XP_012711679.1| PREDICTED: pi...,419,439,20
86,Transcript_100008,gi|831567809|ref|XP_012733833.1| PREDICTED: my...,95,278,183


# Filtered and annotated expresion table, removed contigs with no expression

In [186]:
merged_table.shape

(87271, 14)

In [187]:
merged_table.head()

,Name_x,Length,EffectiveLength,TPM,NumReads,Gene,seqid,Name_y,Note,database,Dbxref,start,end,length
0,TRINITY_DN2268_c0_g1_i1,272,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187,271,84
1,TRINITY_DN2223_c0_g1_i1,272,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2,272,270
2,TRINITY_DN2294_c0_g1_i1,691,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1,85,84
3,TRINITY_DN2293_c0_g1_i1,646,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395,482,87
4,TRINITY_DN73018_c0_g1_i1,425,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0,425,425


# *F. heteroclitus* (RefSeq) annotations, and other annotations, removed contigs with no expression

In [188]:
fhet_merged_table = pd.merge(merged_table,fhet_filtered,on='seqid',how='outer')

In [189]:
fhet_merged_table.shape

(111363, 18)

In [192]:
fhet_merged_table.head(50)

,Name_x,Length,EffectiveLength,TPM,NumReads,Gene,seqid,Name_y,Note,database,Dbxref,start_x,end_x,length_x,Name,start_y,end_y,length_y
0,TRINITY_DN2268_c0_g1_i1,272.0,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187.0,271.0,84.0,NaN,NaN,NaN,NaN
1,TRINITY_DN2223_c0_g1_i1,272.0,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2.0,272.0,270.0,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0.0,90.0,90.0
2,TRINITY_DN2294_c0_g1_i1,691.0,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1.0,85.0,84.0,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0.0,28.0,28.0
3,TRINITY_DN2293_c0_g1_i1,646.0,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395.0,482.0,87.0,NaN,NaN,NaN,NaN
4,TRINITY_DN73018_c0_g1_i1,425.0,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0.0,425.0,425.0,NaN,NaN,NaN,NaN
5,TRINITY_DN73047_c0_g1_i1,204.0,36.667,12.545837,5.413544,TRINITY_DN73047_c0_g1,Transcript_83,SSU_rRNA_eukarya,-,NaN,"""Rfam:RF01960""",0.0,204.0,204.0,NaN,NaN,NaN,NaN
6,TRINITY_DN73096_c0_g1_i1,1196.0,945.347,0.719106,8.000000,TRINITY_DN73096_c0_g1,Transcript_89,SPU_016539-tr,NaN,OrthoDB,NaN,361.0,508.0,147.0,NaN,NaN,NaN,NaN
7,TRINITY_DN73019_c0_g1_i1,1206.0,955.347,0.978421,11.000000,TRINITY_DN73019_c0_g1,Transcript_124,ENSPSIP00000002966,NaN,OrthoDB,NaN,1128.0,1206.0,78.0,NaN,NaN,NaN,NaN
8,TRINITY_DN91680_c0_g1_i1,532.0,286.607,2.371903,8.000000,TRINITY_DN91680_c0_g1,Transcript_152,ENSONIP00000025839,NaN,OrthoDB,NaN,1.0,400.0,399.0,NaN,NaN,NaN,NaN
9,TRINITY_DN91684_c0_g1_i1,963.0,712.780,0.357652,3.000000,TRINITY_DN91684_c0_g1,Transcript_154,ENSRNOP00000014031,NaN,OrthoDB,NaN,0.0,51.0,51.0,NaN,NaN,NaN,NaN


In [193]:
test = fhet_merged_table.drop_duplicates('seqid')
test.shape

(111363, 18)

In [194]:
fhet_merged_table.head()

,Name_x,Length,EffectiveLength,TPM,NumReads,Gene,seqid,Name_y,Note,database,Dbxref,start_x,end_x,length_x,Name,start_y,end_y,length_y
0,TRINITY_DN2268_c0_g1_i1,272.0,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187.0,271.0,84.0,NaN,NaN,NaN,NaN
1,TRINITY_DN2223_c0_g1_i1,272.0,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2.0,272.0,270.0,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0.0,90.0,90.0
2,TRINITY_DN2294_c0_g1_i1,691.0,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1.0,85.0,84.0,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0.0,28.0,28.0
3,TRINITY_DN2293_c0_g1_i1,646.0,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395.0,482.0,87.0,NaN,NaN,NaN,NaN
4,TRINITY_DN73018_c0_g1_i1,425.0,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0.0,425.0,425.0,NaN,NaN,NaN,NaN


In [195]:
fhet_merged_table.columns = ['Contig','Length','EffectiveLength','TPM','NumReads','Gene','dammit.seqid','GeneName','Note','database','Dbxref','start.annotation','end.annotation','length.annotation','FhetNCBIName','FhetNCBI.start','FhetNCBI.end','Fhet.NCBI.length']
fhet_merged_table.head()

,Contig,Length,EffectiveLength,TPM,NumReads,Gene,dammit.seqid,GeneName,Note,database,Dbxref,start.annotation,end.annotation,length.annotation,FhetNCBIName,FhetNCBI.start,FhetNCBI.end,Fhet.NCBI.length
0,TRINITY_DN2268_c0_g1_i1,272.0,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187.0,271.0,84.0,NaN,NaN,NaN,NaN
1,TRINITY_DN2223_c0_g1_i1,272.0,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2.0,272.0,270.0,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0.0,90.0,90.0
2,TRINITY_DN2294_c0_g1_i1,691.0,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1.0,85.0,84.0,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0.0,28.0,28.0
3,TRINITY_DN2293_c0_g1_i1,646.0,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395.0,482.0,87.0,NaN,NaN,NaN,NaN
4,TRINITY_DN73018_c0_g1_i1,425.0,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0.0,425.0,425.0,NaN,NaN,NaN,NaN


# Unique *F. heteroclitus* (RefSeq) annotations, removed contigs with no expression

In [196]:
print(len(fhet_merged_table.FhetNCBIName.unique()))

22632


In [197]:
fhet_merged_table['split1'], fhet_merged_table['split2'],fhet_merged_table['split3'],fhet_merged_table['NCBIproteinID'],fhet_merged_table['NCBIproteinName'] = fhet_merged_table['FhetNCBIName'].str.split('|', 5).str

In [198]:
fhet_merged_table.shape

(111363, 23)

In [199]:
fhet_merged_table.head()

,Contig,Length,EffectiveLength,TPM,NumReads,Gene,dammit.seqid,GeneName,Note,database,...,length.annotation,FhetNCBIName,FhetNCBI.start,FhetNCBI.end,Fhet.NCBI.length,split1,split2,split3,NCBIproteinID,NCBIproteinName
0,TRINITY_DN2268_c0_g1_i1,272.0,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,...,84.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,TRINITY_DN2223_c0_g1_i1,272.0,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,...,270.0,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0.0,90.0,90.0,gi,831524180,ref,XP_012718601.1,PREDICTED: leucine-rich repeat-containing pro...
2,TRINITY_DN2294_c0_g1_i1,691.0,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,...,84.0,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0.0,28.0,28.0,gi,831580603,ref,XP_012738233.1,PREDICTED: abl interactor 2-like isoform X4 [...
3,TRINITY_DN2293_c0_g1_i1,646.0,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,...,87.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,TRINITY_DN73018_c0_g1_i1,425.0,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,...,425.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [200]:
fhet_merged_table = fhet_merged_table[['Contig','Length','EffectiveLength','TPM','NumReads','Gene','dammit.seqid','GeneName','Note','database','Dbxref','start.annotation','end.annotation','length.annotation','FhetNCBIName','FhetNCBI.start','FhetNCBI.end','Fhet.NCBI.length','NCBIproteinName','NCBIproteinID']]

In [201]:
fhet_merged_table.shape

(111363, 20)

In [202]:
fhet_merged_table.head()

,Contig,Length,EffectiveLength,TPM,NumReads,Gene,dammit.seqid,GeneName,Note,database,Dbxref,start.annotation,end.annotation,length.annotation,FhetNCBIName,FhetNCBI.start,FhetNCBI.end,Fhet.NCBI.length,NCBIproteinName,NCBIproteinID
0,TRINITY_DN2268_c0_g1_i1,272.0,66.900,1.347844,1.061129,TRINITY_DN2268_c0_g1,Transcript_4,EHJ73841,NaN,OrthoDB,NaN,187.0,271.0,84.0,NaN,NaN,NaN,NaN,NaN,NaN
1,TRINITY_DN2223_c0_g1_i1,272.0,66.900,2.540396,2.000000,TRINITY_DN2223_c0_g1,Transcript_25,ENSTNIP00000015762,NaN,OrthoDB,NaN,2.0,272.0,270.0,gi|831524180|ref|XP_012718601.1| PREDICTED: le...,0.0,90.0,90.0,PREDICTED: leucine-rich repeat-containing pro...,XP_012718601.1
2,TRINITY_DN2294_c0_g1_i1,691.0,441.742,0.961823,5.000000,TRINITY_DN2294_c0_g1,Transcript_27,ENSXMAP00000008686,NaN,OrthoDB,NaN,1.0,85.0,84.0,gi|831580603|ref|XP_012738233.1| PREDICTED: ab...,0.0,28.0,28.0,PREDICTED: abl interactor 2-like isoform X4 [...,XP_012738233.1
3,TRINITY_DN2293_c0_g1_i1,646.0,397.346,0.855432,4.000000,TRINITY_DN2293_c0_g1,Transcript_34,ENSPTRP00000039483,NaN,OrthoDB,NaN,395.0,482.0,87.0,NaN,NaN,NaN,NaN,NaN,NaN
4,TRINITY_DN73018_c0_g1_i1,425.0,187.049,3.634368,8.000000,TRINITY_DN73018_c0_g1,Transcript_71,LSU_rRNA_eukarya,-,NaN,"""Rfam:RF02543""",0.0,425.0,425.0,NaN,NaN,NaN,NaN,NaN,NaN


In [203]:
print(len(fhet_merged_table.NCBIproteinID.unique()))

22632


In [204]:
print(len(fhet_merged_table.GeneName.unique()))

36696


In [205]:
print(len(fhet_merged_table.Gene.unique()))

43628


In [50]:
# What are the Genes that overlap,
# thinking it's a unique gene, but has overlapping GeneName and NCBIProteinID